# Stats about Train / Dev / Test sets

In [1]:
import pandas as pd

import sys
sys.path.insert(0, '../')
from utils.config import PATHS
from utils.latex import show_latex, TABLES

## Load data

In [2]:
datapath = PATHS.getpath('data_expr_sept')
train = pd.read_pickle(datapath / 'clf_domains/train_excl_bck_add_pilot.pkl')
test = pd.read_pickle(datapath / 'clf_domains/test.pkl')
dev = pd.read_pickle(datapath / 'clf_domains/dev.pkl')

In [3]:
domains=['ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']

## Check correct split

i.e. there are no notes that appear in more than one set

In [4]:
print(test.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(test.NotitieID).any())

False
False
False


## Number of sentences per domain

- A sentence can contain more than one domain and therefore be counted more than once.
- The last column is the total number of sentences in the dataset (incl. all negative examples)

In [5]:
caption = "Domain classification: datasets, sentence-level"
label = "domains_datasets_sents"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'pad_sen_id']]),
    columns = domains,
    data = data.labels.to_list()
)

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_sentences')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_sentences
dataset,,,,,,,,,,
dev,411,22,29,105,225,119,127,96,147,"21,742"
test,775,39,54,160,382,253,287,125,181,"22,082"
train,"4,988",247,486,989,"2,420","2,489","1,967",755,"3,390","239,153"
total,"6,174",308,569,"1,254","3,027","2,861","2,381",976,"3,718","282,977"


In [ ]:
caption = "Domain classification: datasets, note-level"
label = "domains_datasets_notes"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'pad_sen_id']]),
    columns = domains,
    data = data.labels.to_list()
)

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_sentences')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_sentences
dataset,,,,,,,,,,
dev,411,22,29,105,225,119,127,96,147,"21,742"
test,775,39,54,160,382,253,287,125,181,"22,082"
train,"4,988",247,486,989,"2,420","2,489","1,967",755,"3,390","239,153"
total,"6,174",308,569,"1,254","3,027","2,861","2,381",976,"3,718","282,977"


## % of domain sentences out of total 

In [6]:
piv.div(piv.n_sentences, axis=0).mul(100).round(2)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_sentences
dataset,,,,,,,,,,
dev,1.89,0.10,0.13,0.48,1.03,0.55,0.58,0.44,0.68,100.0
test,3.51,0.18,0.24,0.72,1.73,1.15,1.30,0.57,0.82,100.0
train,2.09,0.10,0.20,0.41,1.01,1.04,0.82,0.32,1.42,100.0
total,2.18,0.11,0.20,0.44,1.07,1.01,0.84,0.34,1.31,100.0


In [7]:
prefix = 'ml_datasets'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)